In [12]:
import cv2
from scenedetect import VideoManager
from scenedetect import SceneManager
from scenedetect.detectors import ContentDetector
from datetime import datetime, timedelta
import numpy as np
from moviepy.editor import VideoFileClip

def get_video_duration(video_file_path):
    with VideoFileClip(video_file_path) as video:
        return video.duration  # duration in seconds

def find_subarray_np(main_array, sub_array):
    main_array = np.array(main_array)
    sub_array = np.array(sub_array)
    sub_len = len(sub_array)

    print(main_array)
    print(sub_array)

    strided = np.lib.stride_tricks.sliding_window_view(main_array, window_shape=sub_len)

    matches = np.all(strided == sub_array, axis=1)

    indices = np.where(matches)[0]
    
    if indices.size > 0:
        return indices
    else:
        return -1

# Function to parse timecodes into timedelta objects
def parse_timecode(time_str):
    # Format: 'HH:MM:SS.sss'
    return datetime.strptime(time_str, "%H:%M:%S.%f")

# Function to convert seconds into hh:mm:ss.sss format
def seconds_to_timestamp(seconds):
    td = timedelta(seconds=seconds)
    str_time = str(td)
    hours, minutes, seconds = str_time.split(':')
    seconds, microseconds = seconds.split('.')
    milliseconds = f"{int(microseconds):03d}"[:3]
    return f"{hours}:{minutes}:{seconds}.{milliseconds}"

# Function to compute differences in timestamps
def compute_differences(timestamps):
    times = [parse_timecode(t) for t in timestamps]
    
    differences = []
    for i in range(1, len(times)):
        diff = (times[i] - times[i-1]).total_seconds()
        differences.append(round(diff, 2))
    return differences

def find_scenes(video_path, threshold=30.0):
    shot_boundaries = []

    # Create a video manager object for the video.
    video_manager = VideoManager([video_path])
    scene_manager = SceneManager()
    
    # Add the ContentDetector algorithm (with a threshold setting).
    scene_manager.add_detector(ContentDetector(threshold=threshold))
    
    # Start the video manager and perform scene detection.
    video_manager.set_downscale_factor()
    video_manager.start()

    # Detect scenes and return a list of scenes.
    scene_manager.detect_scenes(frame_source=video_manager)
    
    # Obtain the scenes by frame and timecode.
    scene_list = scene_manager.get_scene_list(video_manager.get_base_timecode())
    
    # Each scene is a tuple of (start, end) FrameTimecodes.
    print('List of scene changes:')
    for i, scene in enumerate(scene_list):
        shot_boundaries.append(scene[1].get_timecode())
        #print(f'Scene {i+1}: Start {scene[0].get_timecode()} - End {scene[1].get_timecode()}')

    video_manager.release()
    return [shot_boundaries, scene_list]

def compute_time_difference(time1, time2):
    datetime1 = parse_timecode(time1)
    datetime2 = parse_timecode(time2)

    difference = datetime1 - datetime2 if datetime1 > datetime2 else datetime2 - datetime1

    return difference.total_seconds()

def get_fps(video_path):
    video = cv2.VideoCapture(video_path)
    fps = video.get(cv2.CAP_PROP_FPS)
    video.release()
    return fps

def timecode_to_frames(timecode, fps):
    time_obj = parse_timecode(timecode)
    total_seconds = time_obj.hour * 3600 + time_obj.minute * 60 + time_obj.second + time_obj.microsecond / 1e6
    frame_number = int(round(total_seconds * fps))
    return frame_number

# Example usage:
video_file_path = '../../dataset/originals/video5.mp4'
query_file_path = '../../dataset/Tests/video5_1_modified.mp4'

original_video_boundaries, original_scenes = find_scenes(video_file_path)
original_video_differences = compute_differences(original_video_boundaries)

query_video_boundaries, query_scenes = find_scenes(query_file_path)
query_video_differences = compute_differences(query_video_boundaries)[:-1]

start_index = find_subarray_np(original_video_differences, query_video_differences)[0]

print(original_video_boundaries)
print(query_video_boundaries)

if(start_index == 0):
    start_time = compute_time_difference(original_video_boundaries[start_index + 1], query_video_boundaries[start_index + 1])
else:
    start_time = compute_time_difference(original_video_boundaries[start_index], query_video_boundaries[0])

fps = get_fps(video_file_path)

query_video_duration = get_video_duration(query_file_path)
end_time = start_time + query_video_duration

start_time = seconds_to_timestamp(start_time)
end_time = seconds_to_timestamp(end_time)
start_frame = timecode_to_frames(start_time, fps)
end_frame = timecode_to_frames(end_time, fps)

print(f"Start Time: {start_time} seconds")
print(f"End Time: {end_time} seconds")
print(f"Start Frame: {start_frame}")
print(f"End Frame: {end_frame}")


VideoManager is deprecated and will be removed.


`base_timecode` argument is deprecated and has no effect.
VideoManager is deprecated and will be removed.


List of scene changes:


`base_timecode` argument is deprecated and has no effect.


List of scene changes:
[ 0.67  2.43  5.57  9.53  2.    2.3   2.47  0.5   1.67  0.67  0.5   0.5
  0.67  0.5   0.5   0.5   0.5   0.5   0.5  16.93 80.23 29.3   1.4   4.73
 24.87 77.97  5.07 61.9   2.3   2.9   4.03  1.27  2.97  7.57  2.33  1.67
  2.6   3.33  3.2   3.63  0.73 13.37  2.73 26.6   4.17  2.47  3.33  1.1
  7.1   9.9  29.2  21.07  1.   19.23  1.47  1.83  5.3  74.    0.63  3.2
  3.    8.77]
[]
['00:00:09.167', '00:00:09.833', '00:00:12.267', '00:00:17.833', '00:00:27.367', '00:00:29.367', '00:00:31.667', '00:00:34.133', '00:00:34.633', '00:00:36.300', '00:00:36.967', '00:00:37.467', '00:00:37.967', '00:00:38.633', '00:00:39.133', '00:00:39.633', '00:00:40.133', '00:00:40.633', '00:00:41.133', '00:00:41.633', '00:00:58.567', '00:02:18.800', '00:02:48.100', '00:02:49.500', '00:02:54.233', '00:03:19.100', '00:04:37.067', '00:04:42.133', '00:05:44.033', '00:05:46.333', '00:05:49.233', '00:05:53.267', '00:05:54.533', '00:05:57.500', '00:06:05.067', '00:06:07.400', '00:06:09.067', '00:0

In [21]:
import cv2
from scenedetect import VideoManager
from scenedetect import SceneManager
from scenedetect.detectors import ContentDetector
from datetime import datetime, timedelta
import numpy as np
from moviepy.editor import VideoFileClip
import json

def get_video_duration(video_file_path):
    with VideoFileClip(video_file_path) as video:
        return video.duration  # duration in seconds

def find_subarray_np(main_array, sub_array):
    main_array = np.array(main_array)
    sub_array = np.array(sub_array)
    sub_len = len(sub_array)

    # print(main_array)
    # print(sub_array)

    strided = np.lib.stride_tricks.sliding_window_view(main_array, window_shape=sub_len)

    matches = np.all(strided == sub_array, axis=1)

    indices = np.where(matches)[0]
    
    if indices.size > 0:
        return indices
    else:
        return [-1]

# Function to parse timecodes into timedelta objects
def parse_timecode(time_str):
    # Format: 'HH:MM:SS.sss'
    return datetime.strptime(time_str, "%H:%M:%S.%f")

# Function to convert seconds into hh:mm:ss.sss format
def seconds_to_timestamp(seconds):
    td = timedelta(seconds=seconds)
    str_time = str(td)
    hours, minutes, seconds = str_time.split(':')
    seconds, microseconds = seconds.split('.')
    milliseconds = f"{int(microseconds):03d}"[:3]
    return f"{hours}:{minutes}:{seconds}.{milliseconds}"

# Function to compute differences in timestamps
def compute_differences(timestamps):
    times = [parse_timecode(t) for t in timestamps]
    
    differences = []
    for i in range(1, len(times)):
        diff = (times[i] - times[i-1]).total_seconds()
        differences.append(round(diff, 2))
    return differences

def find_scenes(video_path, threshold=30.0):
    shot_boundaries = []

    # Create a video manager object for the video.
    video_manager = VideoManager([video_path])
    scene_manager = SceneManager()
    
    # Add the ContentDetector algorithm (with a threshold setting).
    scene_manager.add_detector(ContentDetector(threshold=threshold))
    
    # Start the video manager and perform scene detection.
    video_manager.set_downscale_factor()
    video_manager.start()

    # Detect scenes and return a list of scenes.
    scene_manager.detect_scenes(frame_source=video_manager)
    
    # Obtain the scenes by frame and timecode.
    scene_list = scene_manager.get_scene_list(video_manager.get_base_timecode())
    
    # Each scene is a tuple of (start, end) FrameTimecodes.
    # print('List of scene changes:')
    for i, scene in enumerate(scene_list):
        shot_boundaries.append(scene[1].get_timecode())
        #print(f'Scene {i+1}: Start {scene[0].get_timecode()} - End {scene[1].get_timecode()}')

    video_manager.release()
    return [shot_boundaries, scene_list]

def compute_time_difference(time1, time2):
    datetime1 = parse_timecode(time1)
    datetime2 = parse_timecode(time2)

    difference = datetime1 - datetime2 if datetime1 > datetime2 else datetime2 - datetime1

    return difference.total_seconds()

def get_fps(video_path):
    video = cv2.VideoCapture(video_path)
    fps = video.get(cv2.CAP_PROP_FPS)
    video.release()
    return fps

def timecode_to_frames(timecode, fps):
    time_obj = parse_timecode(timecode)
    total_seconds = time_obj.hour * 3600 + time_obj.minute * 60 + time_obj.second + time_obj.microsecond / 1e6
    frame_number = int(round(total_seconds * fps))
    return frame_number

# Example usage:
# video_file_path = '../../dataset/originals/video4.mp4'
query_file_path = '../../dataset/Tests/video10_1_modified.mp4'

# original_video_boundaries, original_scenes = find_scenes(video_file_path)
# original_video_differences = compute_differences(original_video_boundaries)

query_video_boundaries, query_scenes = find_scenes(query_file_path)
query_video_differences = compute_differences(query_video_boundaries)[:-1]

if(query_video_differences == []):
    print("No Shot Boundaries present in the query video")
else:
    print("Query Video contains Shot Boundaries.\nStarting shot boundary match now:")
    # Read the JSON file
    with open('shotBoundSignature.json', 'r') as file:
        data = json.load(file)

    # Get the arrays from the data
    arrays = data['arrays']

    # Iterate over each array
    for key, array in arrays.items():
        print("Array", key + ":")
        original_video_boundaries = array
        original_video_differences = compute_differences(original_video_boundaries)
        # print(original_video_differences)

        if(len(original_video_differences) < len(query_video_differences)):
            print("No shot Boundary match found\n")
            continue

        start_index = find_subarray_np(original_video_differences, query_video_differences)[0]

        # print(original_video_boundaries)
        # print(query_video_boundaries)

        if(start_index == -1):
            print("No shot Boundary match found\n")
        else:
            print("Shot Boundary match found at index", start_index)
            print(key + " is under consideration\n")
    

    # if(start_index == 0):
    #     start_time = compute_time_difference(original_video_boundaries[start_index + 1], query_video_boundaries[start_index + 1])
    # else:
    #     start_time = compute_time_difference(original_video_boundaries[start_index], query_video_boundaries[0])

    # fps = get_fps(video_file_path)

    # query_video_duration = get_video_duration(query_file_path)
    # end_time = start_time + query_video_duration

    # start_time = seconds_to_timestamp(start_time)
    # end_time = seconds_to_timestamp(end_time)
    # start_frame = timecode_to_frames(start_time, fps)
    # end_frame = timecode_to_frames(end_time, fps)

    # print(f"Start Time: {start_time} seconds")
    # print(f"End Time: {end_time} seconds")
    # print(f"Start Frame: {start_frame}")
    # print(f"End Frame: {end_frame}")


VideoManager is deprecated and will be removed.
`base_timecode` argument is deprecated and has no effect.


No Shot Boundaries present in the query video
